## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
updated_df = application_df.drop(columns=['EIN'])

In [40]:
# Determine the number of unique values in each column.
unique_counts = updated_df.nunique()
print(unique_counts)

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [41]:
# Look at NAME value counts to identify and replace with "Other"
name_counts = updated_df['NAME'].value_counts()

# Display value counts
print(name_counts)

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64


In [67]:
# Choose a cutoff value and create a list of names to be replaced
cutoff = 5
names_to_replace = name_counts[name_counts < cutoff].index.tolist()

# Replace in dataframe
for name in names_to_replace:
    updated_df['NAME'] = updated_df['NAME'].replace(name,"Other")

# Check to make sure replacement was successful
print(updated_df['NAME'].value_counts())

NAME
Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: count, Length: 223, dtype: int64


In [68]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
value_counts = updated_df['APPLICATION_TYPE'].value_counts()

# Display value counts
print(value_counts)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [69]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 528
application_types_to_replace = value_counts[value_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    updated_df['APPLICATION_TYPE'] = updated_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
print(updated_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [70]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_value_counts = updated_df['CLASSIFICATION'].value_counts()
print(class_value_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [71]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts_gt_1 = class_value_counts[class_value_counts > 1]
print(class_value_counts_gt_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [72]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1883
classifications_to_replace = class_value_counts[class_value_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    updated_df['CLASSIFICATION'] = updated_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
print(updated_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [73]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(updated_df)

In [74]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(columns='IS_SUCCESSFUL')
y = application_df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [75]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [110]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# Used BatchNormalization to try and improve performance
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=75, input_dim=266))
nn_model.add(tf.keras.layers.BatchNormalization())
nn_model.add(tf.keras.layers.ReLU())

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=75, input_dim=266))
nn_model.add(tf.keras.layers.BatchNormalization())
nn_model.add(tf.keras.layers.ReLU())

# Added  an additional layer to improve performance
nn_model.add(tf.keras.layers.Dense(units=75, input_dim=266))
nn_model.add(tf.keras.layers.BatchNormalization())
nn_model.add(tf.keras.layers.ReLU())

# Added an additional layer to improve performance
nn_model.add(tf.keras.layers.Dense(units=75, input_dim=266))
nn_model.add(tf.keras.layers.BatchNormalization())
nn_model.add(tf.keras.layers.ReLU())

# Added an additional layer to improve performance
nn_model.add(tf.keras.layers.Dense(units=75, input_dim=266))
nn_model.add(tf.keras.layers.BatchNormalization())
nn_model.add(tf.keras.layers.ReLU())

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_88 (Dense)                     │ (None, 75)                  │          20,025 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_74               │ (None, 75)                  │             300 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_72 (ReLU)                      │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_89 (Dense)                     │ (None, 75)                  │           5,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_75               │ (None, 75)                  │             300 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_73 (ReLU)                      │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 75)                  │           5,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_76               │ (None, 75)                  │             300 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_74 (ReLU)                      │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_91 (Dense)                     │ (None, 75)                  │           5,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_77               │ (None, 75)                  │             300 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_75 (ReLU)                      │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_92 (Dense)                     │ (None, 75)                  │           5,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_78               │ (None, 75)                  │             300 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_76 (ReLU)                      │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_93 (Dense)                     │ (None, 1)                   │              76 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,401 (173.44 KB)

 Trainable params: 43,651 (170.51 KB)

 Non-trainable params: 750 (2.93 KB)

In [111]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [120]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7990 - loss: 0.4162
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7986 - loss: 0.4152
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7979 - loss: 0.4180
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8005 - loss: 0.4140
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8001 - loss: 0.4185
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7968 - loss: 0.4175
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8011 - loss: 0.4103
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7957 - loss: 0.4182
Epoch 9/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7937 - loss: 0.4210
Epoch 10/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8027 - loss: 0.4074
Epoch 11/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8009 - loss: 0.4142
Epoch 12/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step

In [121]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7792 - loss: 0.4802
Loss: 0.48015540838241577, Accuracy: 0.7792419791221619


In [123]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization.h5')